In [0]:
# Databricks notebook source
import uuid
import random
from datetime import datetime, timedelta
import re

# COMMAND ----------

# Unity Catalog 설정
# !!! 실제 Unity Catalog와 스키마 이름으로 변경해 주세요 !!!
CATALOG_NAME = "sudong"
SCHEMA_NAME = "agent_bricks"
TABLE_NAME = "customer_call_transcripts" # 테이블 이름을 필요에 맞게 수정하세요
FULL_TABLE_PATH = f"{CATALOG_NAME}.{SCHEMA_NAME}.{TABLE_NAME}"

# COMMAND ----------

## 대화 데이터 생성을 위한 헬퍼 함수

def _get_timestamp(start_time, seconds_offset):
    """포맷된 타임스탬프를 반환합니다."""
    current_time = start_time + timedelta(seconds=seconds_offset)
    return current_time.strftime("%H:%M:%S")

def _generate_transcript_line(start_time, current_offset, speaker, text):
    """타임스탬프가 포함된 한 줄 대화문을 만듭니다."""
    timestamp = _get_timestamp(start_time, current_offset)
    return f"[{timestamp}] {speaker}: {text}"

# COMMAND ----------

## 대화 시나리오 데이터 구조 (시나리오별 체인)

# 일관된 플레이스홀더용 제품 정보
PRODUCT_NAMES = [
    "플로럴 맥시 드레스", "유기농 면 티셔츠", "슬림핏 데님 진",
    "캐시미어 블렌드 스웨터", "린넨 블렌드 반바지", "클래식 트렌치코트",
    "스포티 조거", "실크 스카프", "울 블렌드 블레이저", "빈티지 워시 후디",
    "스트라이프 버튼업 셔츠", "하이웨이스트 레깅스", "퍼퍼 재킷",
    "플로우 미디 스커트", "니트 가디건", "자수 블라우스"
]
COLORS = ["검정", "흰색", "네이비 블루", "회색", "포레스트 그린", "버건디", "크림", "데님 워시", "로즈 핑크", "올리브 그린", "연파랑", "코랄"]
SIZES = ["XS", "S", "M", "L", "XL", "XXL", "26사이즈", "28사이즈", "30사이즈", "32사이즈", "34사이즈", "36사이즈", "6사이즈", "8사이즈", "10사이즈", "12사이즈", "14사이즈"]

# --- 각 시나리오 및 해결 유형별 대화 체인 ---
# 각 체인은 dict의 리스트: [{"speaker": "고객/상담원", "text": "대화 템플릿"}]
# 체인은 내용에 따라 자연스럽게 해결로 이어집니다.

DIALOGUE_CHAINS = {
    "sizing_fit": {
        "positive": [
            [ # 체인 1: 기본 사이즈 안내, 긍정적 해결
                {"speaker": "고객", "text": "{product_name}을(를) 보고 있는데 사이즈가 잘 맞을지 모르겠어요. 정사이즈인가요?"},
                {"speaker": "상담원", "text": "{product_name}은(는) 대체로 정사이즈로 출시됩니다. 정확한 치수는 상품 페이지의 사이즈표를 참고해 주세요."},
                {"speaker": "고객", "text": "네, 도움이 됐어요! 표를 확인해볼게요. 고마워요!"},
                {"speaker": "상담원", "text": "별말씀을요! 새 {product_name} 예쁘게 입으세요."}
            ],
            [ # 체인 2: 특정 부위 핏, 긍정적 해결
                {"speaker": "고객", "text": "{product_name}의 {body_part} 부분 핏이 궁금해요. 제 치수는 {measurement_example}인치입니다."},
                {"speaker": "상담원", "text": "{measurement_example}인치 {body_part}라면 {product_name}의 {size_recommendation}이(가) 보통 잘 맞으실 거예요. 신축성도 있습니다."},
                {"speaker": "고객", "text": "딱 궁금했던 부분이에요! 정말 고마워요."},
                {"speaker": "상담원", "text": "도움이 되어 기쁩니다!"}
            ],
            [ # 체인 3: 넉넉한 핏 선호, 긍정적 해결
                {"speaker": "고객", "text": "{product_name}은(는) 어떤 사이즈를 사야 할까요? 평소엔 {size_example} 입는데 넉넉한 핏을 좋아해요."},
                {"speaker": "상담원", "text": "{product_name}을(를) 넉넉하게 입으시려면 평소 {size_example}보다 한 치수 크게 선택해 보세요. 맞지 않으면 무료 반품 가능합니다."},
                {"speaker": "고객", "text": "좋은 생각이에요. 그 사이즈로 할게요. 감사합니다!"},
                {"speaker": "상담원", "text": "완벽해요! 좋은 하루 보내세요!"}
            ]
        ],
        "inconclusive": [
            [ # 체인 1: 전화로 정확한 치수 안내 불가, 결론 유보
                {"speaker": "고객", "text": "{product_name}의 {size_example} 사이즈 정확한 치수 알 수 있을까요?"},
                {"speaker": "상담원", "text": "전화로는 정확한 신체 치수 안내가 어렵지만, 웹사이트의 상세 사이즈표가 도움이 될 거예요. 확인해 보셨나요?"},
                {"speaker": "고객", "text": "좀 더 구체적인 안내를 원했는데, 다시 확인해볼게요."},
                {"speaker": "상담원", "text": "바로 해결해드리지 못해 아쉽네요. 표 확인 후 추가 문의 있으면 언제든 연락 주세요."}
            ],
            [ # 체인 2: 사이즈 중간, 두 개 주문 권장, 결론 유보
                {"speaker": "고객", "text": "{product_name} 사이즈가 중간이라 어떤 걸 사야 할지 모르겠어요. 추천해 주실 수 있나요?"},
                {"speaker": "상담원", "text": "두 사이즈 모두 주문해 보시는 걸 추천드려요. 평소 {size_example}와 한 치수 위/아래를 주문하시고 무료 반품을 이용하세요."},
                {"speaker": "고객", "text": "음, 두 개 주문은 좀 번거롭네요."},
                {"speaker": "상담원", "text": "이해합니다. 직접 입어보지 않는 이상 최선의 방법이에요. 반품도 도와드릴 수 있습니다."}
            ]
        ],
        "negative": [
            [ # 체인 1: 맞춤 사이즈 불가, 부정적 해결
                {"speaker": "고객", "text": "제 치수가 특이한데 {product_name} 맞춤 제작이나 조정이 가능한가요?"},
                {"speaker": "상담원", "text": "맞춤 제작은 제공하지 않으며, 고객님의 치수에는 {product_name}의 표준 사이즈가 완벽하게 맞지 않을 수 있습니다."},
                {"speaker": "고객", "text": "아쉽네요. 그럼 맞는 게 없다는 건가요?"},
                {"speaker": "상담원", "text": "죄송하지만 예외는 없습니다. 표준 사이즈만 제공하고 있습니다."}
            ]
        ]
    },
    "product_info": {
        "positive": [
            [ # 체인 1: 소재 및 관리법, 긍정적
                {"speaker": "고객", "text": "{product_name}의 소재가 뭔가요? 세탁은 어떻게 하나요?"},
                {"speaker": "상담원", "text": "{product_name}은(는) {material_example}로 만들어졌으며, {material_property} 특징이 있습니다. 뒤집어서 찬물에 세탁하고 자연 건조를 권장합니다."},
                {"speaker": "고객", "text": "완벽해요, 궁금한 점이 다 해결됐어요! 감사합니다."},
                {"speaker": "상담원", "text": "별말씀을요! 이 정보가 {product_name} 선택에 도움이 되길 바랍니다."}
            ],
            [ # 체인 2: 지속가능성, 긍정적
                {"speaker": "고객", "text": "제품이 윤리적으로 생산되거나 친환경적인가요? {product_name}이 궁금해요."},
                {"speaker": "상담원", "text": "네, 저희는 지속가능성에 힘쓰고 있습니다! {product_name}은(는) {sustainable_material}로 제작되었고, {country_of_origin}에서 공정하게 생산되었습니다."},
                {"speaker": "고객", "text": "정말 좋은 소식이네요! 자세한 안내 감사합니다."},
                {"speaker": "상담원", "text": "더 자세한 내용은 웹사이트의 지속가능성 페이지를 참고해 주세요."}
            ]
        ],
        "inconclusive": [
            [ # 체인 1: 특정 색상 재고 없음, 결론 유보
                {"speaker": "고객", "text": "{product_name}의 {color_example} 색상 있나요? 특정 색상을 찾고 있어요."},
                {"speaker": "상담원", "text": "죄송하지만 {product_name}의 {color_example} 색상은 현재 온라인 재고가 없으며, 입고 일정도 알 수 없습니다. 다른 색상은 어떠신가요?"},
                {"speaker": "고객", "text": "아니요, 꼭 그 색상이 필요해요. 다시 들어오나요?"},
                {"speaker": "상담원", "text": "해당 색상의 입고 일정은 알 수 없습니다. 불편을 드려 죄송합니다."}
            ]
        ],
        "negative": [
            [ # 체인 1: 미등록 색상 주문 불가, 부정적
                {"speaker": "고객", "text": "{product_name}이(가) 마음에 드는데 {color_example} 색상은 온라인에 없네요. 맞춤 주문 가능한가요?"},
                {"speaker": "상담원", "text": "{product_name}은(는) 온라인에 표시된 색상만 제공되며, 현재 {color_example} 색상은 맞춤 주문이 불가합니다."},
                {"speaker": "고객", "text": "아쉽네요. 기대와 달라서 실망입니다."},
                {"speaker": "상담원", "text": "불편을 드려 죄송하지만, 제품 옵션은 정책상 고정되어 있습니다."}
            ]
        ]
    },
    "order_status": {
        "positive": [
            [ # 체인 1: 배송 추적 안내, 긍정적
                {"speaker": "고객", "text": "제 주문 {order_id_example}이(가) 아직 도착하지 않았어요. 배송 추적이 안 돼요."},
                {"speaker": "상담원", "text": "주문 {order_id_example}은(는) {date_example}에 발송되었습니다. 추적 정보는 24~48시간 후에 업데이트될 수 있습니다."},
                {"speaker": "고객", "text": "네, 곧 오겠죠. 확인해주셔서 감사합니다!"},
                {"speaker": "상담원", "text": "별말씀을요! 주문 {order_id_example}의 배송 상황이 확인되어 다행입니다."}
            ],
            [ # 체인 2: 잘못된 상품 수령, 긍정적 해결
                {"speaker": "고객", "text": "주문 {order_id_example}에 잘못된 상품이 왔어요. {product_name_alt1}을(를) 주문했는데 {product_name_alt2}이(가) 왔어요."},
                {"speaker": "상담원", "text": "불편을 드려 죄송합니다! 주문 {order_id_example}의 {product_name_alt1}을(를) 바로 재발송해드리고, 잘못 온 상품은 반송 라벨을 보내드릴게요."},
                {"speaker": "고객", "text": "정말 감사합니다! 바로 처리해주셔서 좋아요."},
                {"speaker": "상담원", "text": "별말씀을요! 바로 처리해드리겠습니다."}
            ]
        ],
        "inconclusive": [
            [ # 체인 1: 배송 지연, 사유 불명확
                {"speaker": "고객", "text": "주문 {order_id_example}이(가) 지연되고 있는데 이유를 알 수 있을까요?"},
                {"speaker": "상담원", "text": "주문 {order_id_example}은(는) {delay_reason}로 인해 지연되고 있습니다. 불편을 드려 죄송하며 최대한 빨리 배송될 수 있도록 하겠습니다."},
                {"speaker": "고객", "text": "그럼 예상 배송일이 있나요?"},
                {"speaker": "상담원", "text": "예상 배송일을 확답드리기 어렵습니다. 이메일로 안내드릴 예정입니다."}
            ]
        ],
        "negative": [
            [ # 체인 1: 배송완료 표시, 미수령, 부정적 해결
                {"speaker": "고객", "text": "주문 {order_id_example}이(가) 배송완료로 나오는데 못 받았어요. 어떻게 해야 하나요?"},
                {"speaker": "상담원", "text": "주문 {order_id_example}은(는) 제공된 주소로 정상 배송된 것으로 확인되어 환불이 어렵습니다."},
                {"speaker": "고객", "text": "말도 안 돼요! 못 받았는데요!"},
                {"speaker": "상담원", "text": "불편을 드려 죄송하지만, 기록상 정상 배송으로 확인되어 정책상 환불이 어렵습니다."}
            ],
            [ # 체인 2: 배송지 변경 불가, 부정적
                {"speaker": "고객", "text": "주문 {order_id_example}의 배송지를 바꿀 수 있나요? 실수했어요."},
                {"speaker": "상담원", "text": "주문 {order_id_example}이(가) 이미 배송 준비 중이라 배송지 변경이 불가합니다. 운송사에 직접 문의해 보실 수 있습니다."},
                {"speaker": "고객", "text": "그럼 방법이 없는 건가요? 그 주소에 없을 거예요."},
                {"speaker": "상담원", "text": "죄송하지만 예외는 없습니다. 운송사에 직접 문의해 주세요."}
            ]
        ]
    },
    "returns_exchanges": {
        "positive": [
            [ # 체인 1: 기본 반품, 긍정적
                {"speaker": "고객", "text": "{order_id_example} 주문의 {product_name}을(를) 반품하려고 해요. 반품 라벨이 필요한가요?"},
                {"speaker": "상담원", "text": "네, '내 주문' 메뉴에서 반품을 신청하시면 반품 라벨을 제공해 드립니다."},
                {"speaker": "고객", "text": "좋아요, 안내 감사합니다!"},
                {"speaker": "상담원", "text": "별말씀을요! 환불은 곧 처리될 예정입니다."}
            ],
            [ # 체인 2: 사이즈 교환, 긍정적
                {"speaker": "고객", "text": "{product_name}을(를) 다른 {size_example}로 교환할 수 있나요? 지금 것은 너무 {fit_issue}해요."},
                {"speaker": "상담원", "text": "네, {product_name}을(를) 다른 {size_example}로 교환 가능합니다. 기존 상품 반품 라벨을 보내드리고, 도착 후 새 사이즈를 발송해 드릴게요."},
                {"speaker": "고객", "text": "완벽해요! 쉽게 처리해주셔서 감사합니다."},
                {"speaker": "상담원", "text": "교환 상품도 예쁘게 입으세요! 더 궁금한 점 있으시면 언제든 말씀해 주세요."}
            ]
        ],
        "inconclusive": [
            [ # 체인 1: 환불 미처리, 결론 유보
                {"speaker": "고객", "text": "{order_id_example} 주문 상품을 반품했는데 아직 환불이 안 됐어요."},
                {"speaker": "상담원", "text": "{order_id_example}의 반품은 {return_received_date}에 접수되었으며, {processing_days} 영업일 내 환불 처리될 예정입니다."},
                {"speaker": "고객", "text": "그럼 좀 더 기다리면 되겠네요."},
                {"speaker": "상담원", "text": "네, 환불 완료 시 이메일로 안내드릴게요."}
            ]
        ],
        "negative": [
            [ # 체인 1: 세일 상품 반품 불가, 부정적
                {"speaker": "고객", "text": "{product_name}이(가) 맞지 않아 반품하려고 하는데 세일 상품이에요."},
                {"speaker": "상담원", "text": "세일 상품({product_name})은 구매 시 안내된 대로 반품/교환이 불가합니다."},
                {"speaker": "고객", "text": "아쉽네요. 기대와 달라서 실망입니다."},
                {"speaker": "상담원", "text": "불편을 드려 죄송하지만, 반품 정책상 예외는 없습니다."}
            ],
            [ # 체인 2: 반품 기간 초과, 부정적
                {"speaker": "고객", "text": "{product_name}을(를) 반품하려는데 배송 후 {days_past_window}일이 지났어요. 가능할까요?"},
                {"speaker": "상담원", "text": "{order_id_example}은(는) 반품 기간({return_window_days}일)이 지나 반품이 불가합니다. 시스템상 {days_past_window}일 경과로 확인됩니다."},
                {"speaker": "고객", "text": "정말 방법이 없나요? 바빠서 기간을 놓쳤어요."},
                {"speaker": "상담원", "text": "죄송하지만 예외는 없습니다. 반품 정책은 명확히 안내되어 있습니다."}
            ],
            [ # 체인 3: 착용 상품 반품 불가, 부정적
                {"speaker": "고객", "text": "{product_name} 반품 조건이 궁금해요. 한 번 입었어요."},
                {"speaker": "상담원", "text": "{product_name}이(가) 착용 또는 손상된 경우 반품이 불가합니다. 이는 정책에 명시되어 있습니다."},
                {"speaker": "고객", "text": "한 번만 입었는데 너무해요!"},
                {"speaker": "상담원", "text": "죄송하지만, 착용 상품은 반품이 불가합니다."}
            ]
        ]
    }
}

# COMMAND ----------

def generate_call_transcript_single_topic_fixed():
    """하나의 주요 주제에 집중된 고객 상담 콜 대화록을 생성합니다. (턴 수 고정)"""
    call_id = str(uuid.uuid4())
    # 일관된 날짜 생성을 위한 기준일 설정
    base_date = datetime(2025, 7, 7) # 예시: 2025년 7월 7일
    call_start_time = base_date - timedelta(days=random.randint(0, 60),
                                                  hours=random.randint(0, 23),
                                                  minutes=random.randint(0, 59))
    country = random.choice(["미국", "캐나다", "영국", "호주", "독일", "프랑스", "스페인", "일본"])

    transcript_lines = []
    current_offset = 0
    company_name = "애니클로딩"
    agent_name = random.choice(["사라", "데이비드", "에밀리", "마이클", "올리비아", "제임스"])

    # 주요 시나리오 하나 선택
    main_scenarios = ["sizing_fit", "product_info", "order_status", "returns_exchanges"]
    scenario_type = random.choice(main_scenarios)

    # 해결 유형 결정 후 해당 체인 선택
    resolution_type = random.choice(["positive", "inconclusive", "negative"])
    available_chains = DIALOGUE_CHAINS[scenario_type].get(resolution_type, [])
    chosen_chain = random.choice(available_chains)

    def add_line(speaker, text_template, params_dict):
        nonlocal current_offset
        formatted_text = text_template.format(**params_dict)
        transcript_lines.append(_generate_transcript_line(call_start_time, current_offset, speaker, formatted_text))
        current_offset += random.randint(10, 50)

    # --- 파라미터 초기화 ---
    params = {
        "product_name": "", "product_name_alt1": "", "product_name_alt2": "",
        "size_example": "", "color_example": "", "order_id_example": "",
        "country_example": "", "body_part": "", "measurement_example": 0,
        "fit_issue": "", "size_recommendation": "", "weather_condition": "",
        "feature": "", 
        "feature_description": "", "fabric_issue": "", "fabric_issue_solution": "",
        "pocket_count": 0, "pocket_type": "", "lining_material": "",
        "style_descriptor1": "", "style_descriptor2": "", "date_example": "",
        "shipping_cost_example": "", "shipping_time_example": "",
        "investigation_time_example": "", "delay_reason": "",
        "expedited_cost_example": "", "expedited_date_example": "",
        "email_example": "", "material_example": "", "material_property": "",
        "sustainable_material": "", "country_of_origin": "",
        "company_name": company_name, "agent_name": agent_name,
        "return_window_days": 0, "return_received_date": "", "processing_days": 0,
        "days_to_wait": 0, "days_past_window": 0,
        "policy_issue": "", "term_example": "" 
    }

    params.update({
        "product_name": random.choice(PRODUCT_NAMES),
        "size_example": random.choice(SIZES),
        "color_example": random.choice(COLORS),
        "order_id_example": f"ORD{random.randint(100000, 999999)}",
        "country_example": country,
        "body_part": random.choice(["가슴", "허리", "엉덩이", "허벅지", "어깨"]),
        "measurement_example": random.randint(28, 45),
        "fit_issue": random.choice(["너무 꽉 끼어요", "너무 헐렁해요", "너무 짧아요", "너무 길어요"]),
        "size_recommendation": random.choice(SIZES),
        "weather_condition": random.choice(["추움", "따뜻함", "비옴", "맑음", "온화함"]),
        "feature": random.choice(["가벼움", "따뜻함", "스타일리시함", "다용도"]), 
        "feature_description": random.choice(["가벼움", "통기성", "방수", "보온성"]),
        "fabric_issue": random.choice(["보풀", "수축", "변색", "늘어남"]),
        "fabric_issue_solution": random.choice(["보풀 방지", "수축 최소화"]),
        "pocket_count": random.randint(0, 4),
        "pocket_type": random.choice(["실용 포켓", "지퍼 포켓", "사이드 포켓", "프론트 포켓"]),
        "lining_material": random.choice(["폴리에스터", "면", "실크 블렌드"]),
        "style_descriptor1": random.choice(["클래식", "모던", "캐주얼", "우아함"]),
        "style_descriptor2": random.choice(["릴랙스드", "구조적", "미니멀", "보헤미안"]),
        "date_example": (base_date - timedelta(days=random.randint(1, 20))).strftime("%m월 %d일"),
        "shipping_cost_example": f"₩{random.choice(['5,990', '7,500', '9,990'])}",
        "shipping_time_example": random.choice(['3-5', '5-7', '7-10']),
        "investigation_time_example": random.choice(['영업일 기준 3-5일', '최대 일주일']),
        "delay_reason": random.choice(['기상 악화', '통관 지연', '예상치 못한 물량']),
        "expedited_cost_example": f"₩{random.choice(['15,000', '19,990', '25,000'])}",
        "expedited_date_example": (base_date + timedelta(days=random.randint(1, 3))).strftime("%m월 %d일"),
        "email_example": "customer@example.com",
        "material_example": random.choice(["유기농 면", "재활용 폴리에스터", "텐셀", "린넨 블렌드", "비스코스"]),
        "material_property": random.choice(["통기성", "내구성", "부드러움", "구김 방지", "항균성"]),
        "sustainable_material": random.choice(["재활용 면", "유기농 헴프", "책임감 있는 울"]),
        "country_of_origin": random.choice(["포르투갈", "터키", "베트남", "인도", "방글라데시", "중국"]),
        "return_window_days": random.choice([14, 30, 45]),
        "return_received_date": (base_date - timedelta(days=random.randint(5, 20))).strftime("%m월 %d일"),
        "processing_days": random.randint(3, 10),
        "days_to_wait": random.randint(5,10),
        "days_past_window": random.randint(1, 15),
        "policy_issue": random.choice(["최종 세일 상품", "반품 기간", "착용 상품", "원래 태그"]), 
        "term_example": random.choice(["'할인 상품은 최종 판매입니다'", "'반품은 30일 이내에 신청해야 합니다'", "'상품은 미착용, 미세탁 상태여야 합니다'"])
    })
    temp_product_names_for_alts = [p for p in PRODUCT_NAMES if p != params["product_name"]]
    params["product_name_alt1"] = random.choice(temp_product_names_for_alts) if temp_product_names_for_alts else PRODUCT_NAMES[0]
    temp_product_names_for_alts_2 = [p for p in temp_product_names_for_alts if p != params["product_name_alt1"]]
    params["product_name_alt2"] = random.choice(temp_product_names_for_alts_2) if temp_product_names_for_alts_2 else PRODUCT_NAMES[0]

    # --- 대화 시작 ---
    add_line("고객", "안녕하세요, 문의드릴 게 있어요.", params)
    add_line("상담원", "안녕하세요, 무엇을 도와드릴까요?", params)

    # --- 메인 대화 (선택된 체인 사용) ---
    for interaction in chosen_chain:
        add_line(interaction["speaker"], interaction["text"], params)

    # 마지막 인사 (옵션)
    if random.random() < 0.8:
        add_line("상담원", "언제든 도움이 필요하시면 말씀해 주세요. 좋은 하루 보내세요!", params)

    return {
        "call_id": call_id,
        "timestamp": call_start_time,
        "country": country,
        "transcription": "\n".join(transcript_lines)
    }

# COMMAND ----------

## 콜 대화록 100개 생성

num_transcripts = 100
call_data = []

print(f"{num_transcripts}개의 단일 주제 콜 대화록을 생성하고 있습니다...")
for i in range(num_transcripts):
    call_data.append(generate_call_transcript_single_topic_fixed())
    if (i + 1) % 10 == 0:
        print(f"{i + 1}개 생성 완료...")
print("생성이 완료되었습니다!")

# COMMAND ----------

## Spark DataFrame 생성

df = spark.createDataFrame(call_data)
display(df)

# COMMAND ----------

## Unity Catalog에 테이블 생성 및 저장

print(f"카탈로그 '{CATALOG_NAME}'와 스키마 '{SCHEMA_NAME}'를 확인합니다...")
spark.sql(f"CREATE CATALOG IF NOT EXISTS {CATALOG_NAME}")
spark.sql(f"USE CATALOG {CATALOG_NAME}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {SCHEMA_NAME}")
spark.sql(f"USE SCHEMA {SCHEMA_NAME}")

print(f"Unity Catalog 테이블에 데이터를 저장합니다: {FULL_TABLE_PATH}...")

df.write.mode("overwrite").saveAsTable(FULL_TABLE_PATH)

print(f"테이블 '{FULL_TABLE_PATH}'이(가) Unity Catalog에 성공적으로 생성 또는 업데이트되었습니다!")
print("이제 SQL로 쿼리하거나 Spark로 추가 분석이 가능합니다.")

# COMMAND ----------

## 테이블 검증 (옵션)

print(f"'{FULL_TABLE_PATH}'의 데이터를 확인합니다...")
verified_df = spark.read.table(FULL_TABLE_PATH)
print(f"테이블 행 개수: {verified_df.count()}")
display(verified_df.limit(5)) # 일부 행을 표시하여 확인